In [ ]:
# Writing a classifier for images into animal Classes using CIFAR10 data

import torch
import torct.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# device configuration
device = torch.device('cuda' if torch.cuda.is-available() else 'cpu')

# Hyper-params
num_epochs = 4
batch_size = 4
learning_rate = 0.001

# dataset has PILImage images of range[0,1].
# We transform them to Tensors of normalized range[-1,1]

transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform= transform)

test_dataseet = torchvision.datasets.CIFAR1-(root = './data', train = False, download = True, transform = transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle= False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#Convolutional Network
class ConvNet(nn.Module):
    def __init__(self):
        
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 6, 5) # (Input channel size: 3 because RGB, output = 6, ksize = 5)
        self.pool = nn.MaxPool2d(2, 2) #(ksize = 2, stride = 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        pass
    
    def forward(self, x):
        pass

    
    
    

model = ConvNet().to(device)

#Classification, so our loss function will be cross entropy loss, softmax included inside
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


# Training Part
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
#         origin shape: []
        images = images.to(device)
        labels = labels.to(device)
        
        #forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        #Backward pass amd optimize
        optimizer.sero_grad() # this empties the gradients
        loss.backward()
        opeimizer.step()
        
        if (i+1) % 2000 ==0:
            print(f'Epoch [{epoch+1}/{num_epochs}), Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
            
print('Finished Training')


# Evaluating the model

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    
    for images, lables in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        #max returns (value, index)
        _, presicted = torch.max(outputs, 1)
        n_samples += lables.size(0)
        n_correct += (predicted == lables).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if(label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
            
        acc = 100.0 * n_correct / n_samples
        print(f'Accuracy of the network: {acc}%')